# Face Detection and Recognition
This notebook provides a quick resource for exploring OpenCV's built-in face detection and recognition algorithms, and is inspired by the third section of the following youtube video: https://www.youtube.com/watch?v=oXlwWbU8l2o

Original code for the course can be found on the [github profile](https://github.com/jasmcaus/opencv-course) of the course instructor, Jason Dsouza.

Images of 'famous' people are used as examples, which were downloaded from the Labelled Faces in the Wild (LFW) dataset, available as part of the following Kaggle competition: https://www.kaggle.com/jessicali9530/lfw-dataset

The goal of Face Detection is to identify a rectangular bounding box around each face in an input image. Additional features may be detected as well, e.g. eyes, smile, etc. The goal of Face Recognition is to correctly identify the person based on their facial features. Common practice is to first apply face detection, then apply a facial recognition algorithm to identify the name of the person from features in the detected face image.

## References
- https://github.com/Kaggle/kaggle-api (you can use the api to download the LFW data)
- https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
- https://towardsdatascience.com/a-dog-detector-and-breed-classifier-4feb99e1f852 (also see https://github.com/HenryDashwood/dog_breed_classifier)
- https://keras.io/guides/transfer_learning/
- https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
- https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
- https://stackoverflow.com/questions/20801015/recommended-values-for-opencv-detectmultiscale-parameters
- https://docs.opencv.org/4.1.0/d5/d54/group__objdetect.html
- https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/
- 

In [21]:
import cv2 as cv
import numpy as np
import os
import random
from opencv_tools import load_frame_gray, show_frame, detect_primary_objects, detect_image_objects, draw_detected_objects

In [3]:
# Location of base directory containing subfolders with image files for each person
dir_root = "/home/fdpearce/Documents/Projects/data/Images/LFW/lfw-deepfunneled/lfw-deepfunneled"
# Name must match folder name exactly after a single modification: spaces, " ", are converted to underscores, "_" (see get_img_path for details)
people = ["George W Bush", "Laura Bush", "Vladimir Putin", "Gloria Macapagal Arroyo", "Arnold Schwarzenegger", "Megawati Sukarnoputri", \
          "Hugo Chavez", "Serena Williams", "Colin Powell", "Junichiro Koizumi", "Jennifer Capriati"]
num_people_with_most_img = 25

In [4]:
# Print high-level stats, including the number of people with image data, and a list of people with the
# most images, plus the # of Images. num_people_with_most_img determines how many people to display results for
people_folder = os.listdir(dir_root)
num_images_per_folder = [len(os.listdir(os.path.join(dir_root, p))) for p in people_folder]
sort_people_num_img = sorted(zip(people_folder, num_images_per_folder), reverse=True, key=lambda pair: pair[1])
print(f"# of People with Images Available = {len(people_folder)}")
print(f"Top {num_people_with_most_img} People With Most Images, # of Images")
print(*sort_people_num_img[:num_people_with_most_img], sep="\n")

# of People with Images Available = 5749
Top 25 People With Most Images, # of Images
('George_W_Bush', 530)
('Colin_Powell', 236)
('Tony_Blair', 144)
('Donald_Rumsfeld', 121)
('Gerhard_Schroeder', 109)
('Ariel_Sharon', 77)
('Hugo_Chavez', 71)
('Junichiro_Koizumi', 60)
('Jean_Chretien', 55)
('John_Ashcroft', 53)
('Serena_Williams', 52)
('Jacques_Chirac', 52)
('Vladimir_Putin', 49)
('Luiz_Inacio_Lula_da_Silva', 48)
('Gloria_Macapagal_Arroyo', 44)
('Arnold_Schwarzenegger', 42)
('Jennifer_Capriati', 42)
('Laura_Bush', 41)
('Lleyton_Hewitt', 41)
('Hans_Blix', 39)
('Alejandro_Toledo', 39)
('Nestor_Kirchner', 37)
('Andre_Agassi', 36)
('Alvaro_Uribe', 35)
('Megawati_Sukarnoputri', 33)


In [22]:
# Helper functions for creating training data and visualizing/validating results
# using functions from opencv_tools imported above
def get_img_path(dir_root, person, img_num, img_ext):
    """Returns the full path to an image by joining the parameters in their input order:
    dir_root = directory root, one level above folders for individual people that contain image files
    person = person the images are of in a specific subfolder, with spaces between first, last, etc
    img_num = a string identifying the individual image of the person
    img_ext = file extension of the image files
    Image format is roughly dir_root/person/person+_+img_num+_+img_ext, see code for details
    """
    person_fname = "_".join(person.split(" "))
    file_name = "_".join([person_fname, img_num])+img_ext
    return os.path.join(dir_root, person_fname, file_name)

def create_training_data(dir_root, people, person_img_max, detect_params, detect_objects=False, detect_type="all", random_seed=0, verbose=False):
    """Create training data from the following inputs:
    dir_root = a string containing base directory with subfolders containing images for each person
    people = a list containing string values, with each string containing a person's name. These are the folder names, EXCEPT for spaces instead of underscores
    person_img_max = an integer specifying the maximum number of training images to return for each person. Set this to a value larger than that
                     maximum # of images available if you want to process all images
    haar_path = a string containing the path to the haar cascade class definition xml file. Files for different features (e.g. face, eyes, etc) can
                be downloaded from the following link: https://github.com/opencv/opencv/blob/master/data/haarcascades/
    detect_params = a dictionary containing two sets of parameters: 1) 'haar_file', a string specifying the full path to the haar cascade
                    xml file to load and 2) 'params' dict to pass to the detectMultiScale method of the haar cascade class. Valid values
                    include scaleFactor (default=1.1), minNeighbors (default=3), and minSize. 
                    The haar_file in 1) can be downloaded from here: https://github.com/opencv/opencv/blob/master/data/haarcascades/
    detect_objects = a boolean-like value (True/False, 1/0, etc) that turns on/off object detection. Set to false to conduct a dry run that checks
                     how many image files will be processed for each person
    detect_type = an optional string specifying the type of detection to perform:
                  "all": runs detect_all_objects, which returns all objects detected from one execution of the haar class detectMultiScale
                  method with the input parameters specified in detect_params. The number of objects detected may vary greatly from image to
                  image for a fixed set of input parameters
                  "primary": runs detect_primary_objects, which performs an iterative process to return a user-specified number of primary objects
                  detected in the input image. Essentially, the minNeighbors parameter is adjusted until the desired number of objects are detected
    random_seed = an integer that determines the order of the random shuffle applied to images before selecting the training samples
    verbose = a boolean-like value that, when truthy, prints additional details during execution for validation/debugging purposes
    Output is a tuple with two values:
    features = a list containing zero or more lists, with each list containing four values (x, y, w, h) that define the rectangle containing the detected object
    labels = a list containing zero or more integer values, with each int specifying the index to a specific person in the input list of people used for training
    """
    random.seed(random_seed)
    features = []
    labels = []
    for person in people:
        label = people.index(person)
        path = os.path.join(dir_root, "_".join(person.split(" ")))
        img_files = os.listdir(path)
        random.shuffle(img_files)
        img_num = 0
        total_detected = 0
        num_detected = 0
        for img_f in img_files[:min(person_img_max, len(img_files))]:
            img_path = os.path.join(path, img_f)
            img_num += 1
            print(f"Working on Image # {img_num}: {img_f}")
            if detect_objects:
                _, gray = load_frame_gray(img_path, gray_flag=True)
                detected_features, detected_labels = detect_image_objects(gray, detect_params, detect_type=detect_type, label=label, verbose=verbose)
                num_detected = len(detected_features)
                if num_detected:
                    total_detected += num_detected
                    features.extend(detected_features)
                    labels.extend(detected_labels)
        print(f"{img_num} training images with {total_detected} objects identified for {person}")
    return features, labels

def show_person_images(dir_root, person, img_nums, img_ext, object_rectangles, rect_color=(0, 0, 0)):
    """Loop through each each image in the input person's subfolder, whose number is provided in the input list of
    strings, img_nums. Show all objects in the input list of objects, object_rectangles.
    """
    for ind, img_n in enumerate(img_nums):
        img_path = get_img_path(dir_root, person, img_n, img_ext)
        _, gray = load_frame_gray(img_path, gray_flag=True)
        gray = draw_detected_objects(gray, object_rectangles[ind], print_detected=True, frame_to_show=None, rect_color=rect_color)
        show_frame(gray, f"Primary Objects(s) Detected for {person}")

### Load an Example Image and Convert to Grayscale

In [6]:
people = ["Megawati Sukarnoputri"] # for testing purposes
person = people[0]
img_num = "0017"
#img_num = "0027"
img_ext = ".jpg"

In [7]:
# Parse parameters to build path to a single image, img_path, then load grayscale image
per_fname = "_".join(person.split(" "))
img_path = os.path.join(dir_root, per_fname, "_".join([per_fname, img_num])+img_ext)
_, gray_face = load_frame_gray(img_path, gray_flag=True)
print(f"Size of Example Grayscale Image = {gray_face.shape}")

Size of Example Grayscale Image = (250, 250)


## Face Detection

The first step is to download Haar Cascades xml file for frontal face detection from the following link:

https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

If you want to follow the code below, then you should also download the eye and smile detection xml files from the haarcascades folder.

In [8]:
haar_cascade_params = {
    'face': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_frontalface.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'eye': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_eye.xml',
        'num_primary_obj': 2,
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
            'minSize': (20, 20)
        }
    },
    'smile': {
        'haar_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_smile.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 8,
            'minSize': (20, 20)
        }     
    }
}
haar_image = gray_face.copy()

In [9]:
# Face detection
detection_obj = 'face'
haar_obj = haar_cascade_params[detection_obj]
detected_face_rects = detect_primary_objects(haar_image, verbose=True, **haar_obj)

Initial # of Objects Detected = 1
Final # of Objects Detected = 1


In [10]:
# Eye detection
detection_obj = 'eye'
haar_obj = haar_cascade_params[detection_obj]
detected_eye_rects = detect_primary_objects(haar_image, verbose=True, **haar_obj)

Initial # of Objects Detected = 2
Final # of Objects Detected = 2


In [11]:
# Smile detection
detection_obj = 'smile'
haar_obj = haar_cascade_params[detection_obj]
detected_smile_rects = detect_primary_objects(haar_image, verbose=True, **haar_obj)

Initial # of Objects Detected = 4
Iteration # = 1
minNeighbors = 10
Iteration # = 2
minNeighbors = 14
Iteration # = 3
minNeighbors = 16
Iteration # = 4
minNeighbors = 20
Iteration # = 5
minNeighbors = 26
Iteration # = 6
minNeighbors = 34
Final # of Objects Detected = 1


In [12]:
print(f"# of Faces Found = {len(detected_face_rects)}")
print(f"# of Eyes Found = {len(detected_eye_rects)}")
print(f"# of Smiles Found = {len(detected_smile_rects)}")

# of Faces Found = 1
# of Eyes Found = 2
# of Smiles Found = 1


In [13]:
# Print detected smile(s) coordinates, and optionally show on original image
haar_image = draw_detected_objects(haar_image, detected_smile_rects, print_detected=True, frame_to_show=None, rect_color=(255, 0, 0))

Object 0 Location: x=95, y=142, w=58, h=29


In [14]:
# Print detected eye(s) coordinates, and optionally show on original image
haar_image = draw_detected_objects(haar_image, detected_eye_rects, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

Object 0 Location: x=91, y=99, w=29, h=29
Object 1 Location: x=131, y=99, w=30, h=30


In [15]:
# Print detected face(s) coordinates, and optionally show on original image
haar_image = draw_detected_objects(haar_image, detected_face_rects, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

Object 0 Location: x=68, y=67, w=119, h=119


In [16]:
show_frame(haar_image, f"Faces(s) Detected in Image of {person}")

### Face Detection on Training Data
First perform face detection using detection type of 'all', i.e. detection parameters are fixed to those provided at input and all detected objects are returned

In [17]:
person_img_max = 30
detect_obj = 'face'
detect_type = 'all'
haar_obj = haar_cascade_params[detect_obj]
features, labels = create_training_data(dir_root, people, person_img_max, haar_obj, detect_objects=True, detect_type=detect_type, verbose=True)
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

Working on Image # 1: Megawati_Sukarnoputri_0020.jpg
# of Objects Detected = 1
66, 68, 119, 119
Working on Image # 2: Megawati_Sukarnoputri_0006.jpg
# of Objects Detected = 1
73, 73, 104, 104
Working on Image # 3: Megawati_Sukarnoputri_0021.jpg
# of Objects Detected = 1
68, 70, 115, 115
Working on Image # 4: Megawati_Sukarnoputri_0028.jpg
# of Objects Detected = 1
66, 69, 117, 117
Working on Image # 5: Megawati_Sukarnoputri_0027.jpg
# of Objects Detected = 2
7, 49, 102, 102
74, 69, 109, 109
Working on Image # 6: Megawati_Sukarnoputri_0005.jpg
# of Objects Detected = 1
69, 65, 113, 113
Working on Image # 7: Megawati_Sukarnoputri_0016.jpg
# of Objects Detected = 1
67, 69, 114, 114
Working on Image # 8: Megawati_Sukarnoputri_0023.jpg
# of Objects Detected = 1
70, 71, 110, 110
Working on Image # 9: Megawati_Sukarnoputri_0029.jpg
# of Objects Detected = 1
66, 70, 115, 115
Working on Image # 10: Megawati_Sukarnoputri_0012.jpg
# of Objects Detected = 1
53, 54, 133, 133
Working on Image # 11: 

In [18]:
# Visually inspect example images and the bounding box detected by the face recognition algorithm
# Use the verbose setting above to print out bounding boxes
person = people[0]
img_nums = ["0003", "0019", "0025", "0027", "0031"]
img_ext = ".jpg"
# Megawati Sukarnoputri: 'all' examples where 2 faces detected instead of one
face_rectangles = [[[92, 46, 86, 86], [70, 77, 107, 107]], \
                   [[69, 69, 112, 112], [86, 161, 80, 80]], \
                   [[3, 10, 74, 74], [67, 68, 115, 115]], \
                   [[7, 49, 102, 102], [74, 69, 109, 109]], \
                   [[162, 8, 56, 56], [65, 68, 117, 117]]]
show_person_images(dir_root, person, img_nums, img_ext, face_rectangles, rect_color=(0, 0, 0))

Object 0 Location: x=92, y=46, w=86, h=86
Object 1 Location: x=70, y=77, w=107, h=107
Object 0 Location: x=69, y=69, w=112, h=112
Object 1 Location: x=86, y=161, w=80, h=80
Object 0 Location: x=3, y=10, w=74, h=74
Object 1 Location: x=67, y=68, w=115, h=115
Object 0 Location: x=7, y=49, w=102, h=102
Object 1 Location: x=74, y=69, w=109, h=109
Object 0 Location: x=162, y=8, w=56, h=56
Object 1 Location: x=65, y=68, w=117, h=117


In [19]:
# Now perform face detection using detection type of 'primary' for comparison
detect_type = 'primary'
haar_obj = haar_cascade_params[detect_obj]
features, labels = create_training_data(dir_root, people, person_img_max, haar_obj, detect_objects=True, detect_type=detect_type, verbose=True)
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

Working on Image # 1: Megawati_Sukarnoputri_0020.jpg
Initial # of Objects Detected = 1
Final # of Objects Detected = 1
66, 68, 119, 119
Working on Image # 2: Megawati_Sukarnoputri_0006.jpg
Initial # of Objects Detected = 1
Final # of Objects Detected = 1
73, 73, 104, 104
Working on Image # 3: Megawati_Sukarnoputri_0021.jpg
Initial # of Objects Detected = 1
Final # of Objects Detected = 1
68, 70, 115, 115
Working on Image # 4: Megawati_Sukarnoputri_0028.jpg
Initial # of Objects Detected = 1
Final # of Objects Detected = 1
66, 69, 117, 117
Working on Image # 5: Megawati_Sukarnoputri_0027.jpg
Initial # of Objects Detected = 2
Iteration # = 1
minNeighbors = 6
Iteration # = 2
minNeighbors = 10
Iteration # = 3
minNeighbors = 16
Iteration # = 4
minNeighbors = 24
Final # of Objects Detected = 1
74, 69, 109, 109
Working on Image # 6: Megawati_Sukarnoputri_0005.jpg
Initial # of Objects Detected = 1
Final # of Objects Detected = 1
69, 65, 113, 113
Working on Image # 7: Megawati_Sukarnoputri_0016.

In [20]:
face_rectangles = [[[70, 77, 107, 107]], \
                   [[69, 69, 112, 112]], \
                   [[67, 68, 115, 115]], \
                   [[74, 69, 109, 109]], \
                   [[65, 68, 117, 117]]]
show_person_images(dir_root, person, img_nums, img_ext, face_rectangles, rect_color=(0, 0, 0))

Object 0 Location: x=70, y=77, w=107, h=107
Object 0 Location: x=69, y=69, w=112, h=112
Object 0 Location: x=67, y=68, w=115, h=115
Object 0 Location: x=74, y=69, w=109, h=109
Object 0 Location: x=65, y=68, w=117, h=117


### 2. Train Face Recognition Model

In [ ]:
save_recognizer_data = False
face_recognizer = cv.face.LBPHFaceRecognizer_create()

In [ ]:
features = np.array(features, dtype="object")
labels = np.array(labels)

In [ ]:
face_recognizer.train(features, labels)

In [ ]:
if save_recognizer_data:
    face_recognizer.save("face_recognizer.yml")
    np.save("features.npy", features)
    np.save("labels.npy", labels)